# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [12]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [13]:
import requests
import bs4
from bs4 import BeautifulSoup

In [1]:
## YOUR CODE HERE

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

In [2]:
## YOUR CODE HERE

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [7]:
## YOUR CODE HERE

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [8]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [9]:
## YOUR CODE HERE

### Save your results as a CSV

In [14]:
## YOUR CODE HERE

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [26]:
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import classification_report

print(os.getcwd())

indeed = pd.read_csv('/Users/HudsonCavanagh/GA_dsi-projects/projects/4th/indeed_ds_data30_dedupe.csv')
# sample_indeed = pd.read_csv('/Users/HudsonCavanagh/GA_dsi-projects/projects/4th/indeed_ds_data6_first.csv')

/Users/HudsonCavanagh/GA_dsi-projects/projects/4th


In [31]:
indeed = indeed.iloc[:,1:]
indeed.head()

,location,salary,title,hourly_salary,monthly_salary,annual_salary
0,"Topsham, ME",NaN,Senior Research Analyst,NaN,NaN,NaN
1,United States,NaN,Biomedical Laboratory Officer,NaN,NaN,NaN
2,United States,NaN,Software Development Engineer – New AWS Machin...,NaN,NaN,NaN
3,"Lisbon, ME",NaN,Big Data Engineer,NaN,NaN,NaN
4,"Brewer, ME",120000.0,Data Analyst II/III,NaN,NaN,120000.0


In [34]:
type(indeed['salary'][4])

str

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [29]:
annual_avg_sal = indeed['annual_salary'].mean()
#need assumption for hours of work annually...
indeed['salary'] = (indeed['monthly_salary']*12)
indeed['salary'] = (indeed['hourly_salary']*12*5*9)#12 month, 5 days/ week, 9 hours/day
#sample_indeed['salary'] = (sample_indeed['annual_salary'] -- should be accounted for already
avg_sal = indeed['salary'].mean()

print("all data:", avg_sal,"annual:",annual_avg_sal)

#based on the below, we should only use annual_salary figures for this purpose

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [21]:
# annual_avg_sal_sam = sample_indeed['annual_salary'].mean()
# #need assumption for hours of work annually...
# sample_indeed['salary'] = (sample_indeed['monthly_salary']*12)
# sample_indeed['salary'] = (sample_indeed['hourly_salary']*12*5*9)#12 month, 5 days/ week, 9 hours/day
# #sample_indeed['salary'] = (sample_indeed['annual_salary'] -- should be accounted for already
# avg_sal_samp = sample_indeed['salary'].mean()
# print("all data:", avg_sal_samp,"annual:",annual_avg_sal_sam)

# #('all data:', 23331.471428571425, 'annual:', 123773.40909090909)

('all data:', 23331.471428571425, 'annual:', 123773.40909090909)


#### Thought experiment: What is the baseline accuracy for this model?

In [16]:
#50% -- because we're using the average

Thoughts for features

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [36]:
import statsmodels.api as sm
import pandas as pd
import pylab as pl
import numpy as np
import patsy

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score




X_basic = results['location']
y_basic = results['annual_salary'].values #based on scaling not working


X_trainb, X_testb, y_trainb, y_testb = train_test_split(X_basic, y_basic, test_size=0.33, random_state=50)




NameError: name 'results' is not defined

In [ ]:
lr_basic = LogisticRegression(C=1.5, penalty='l1', solver='liblinear') #can i take away penalty? make sure this is statsmodels
lr_basic_model = lr_basic.fit(X_trainb, y_trainb)
lr_basic_ypred = lr_basic_model.predict(X_testb)


#dmmatrices is another technique that does both these things in 1

burg_drug = pd.DataFrame(X1, columns=X1.design_info.column_names)
burg_drug['Target'] 
burg_drug

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.33, random_state=77)

lr_basic_cm = confusion_matrix(y_testb, lr_basic_ypred, labels=lr_basic.classes_)
lr_basic_cm_df = pd.DataFrame(lr_basic_cm, columns=lr_basic.classes_, index=lr_basic.classes_.classes_) #lr_l1_SF is instance of model
lr_basic_cm_df

#from http://localhost:8888/notebooks/dsi-projects/week04/w4d4_advanced_model_tuning_gridsearch_worksheet.ipynb

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [ ]:
indeed_basic

In [11]:
#cost of living estimate, will get to this only if 

import os
import pandas as pd

os.getcwd()

cost_living = pd.read_csv('/Users/HudsonCavanagh/GA_dsi-projects/projects/4th/cost_of_living_index.csv')
cost_living.head()
cost = cost_living
cost.index = cost['City']
cost = cost['Price Index']
cost.columns = ['price_index']
cost.head(10)
default_cost_living = cost.mean()
print(default_cost_living)

173.709302326


In [ ]:
skill_list = ['python', 'r', 'learning', 'SQL', 'visualiz', 'mining', 'mung', 'clean', 'scikit-learn']
big_list = ['big', 'advanced', 'volume', 'hadoop', 'spark', 'engineer', 'large']
senior_list = ['senior', 'sr', 'Sr.', 'lead', 'principle', 'chief', 'specialist']
indeed['senior'] = 0
indeed['skills'] = 0
indeed['skills'] = 0

for index, value in enumerate(indeed['description']):
    if any str(value).split() in senior_list:
        indeed['senior'][index] = 1
    if any str(value).split() in skill_list:
        indeed['skills'][index]=1
    if any str(value).split() in big_list:
        indeed['big_data'][index]=1


In [ ]:
pd.merge()

indeed_mod = indeed.merge(cost, how='left', on=None, left_on='location', right_index=True)

ideas for features:
####location
-proxy for region
-find cost of living by city (estimate)
-major hub proxy (SF, NYC)
-fourtune 100?

####role title (if the following words in description)
-senior (or sr. or sr)
-junior (or jr. or jr)
-python
-r
-tableau
-Machine Learning
-Software Develop
-volume
-big/ large/ 
-advanced
-predictive
-lead
-principle
-analytic
-insight
-SQL
-finance/ financial
-visualization
-model
-optimize
-specialist
-mining/ munging/ cleaning
-scraping



In [ ]:
X1 = patsy.dmatrix('~ C(hour) + C(DayOfWeek) + C(PdDistrict)', sf_crime_small) #dmatrix uses formula to output independ vars
y1 = sample_indeed['salary'].values #set target


lr_l1_SF = LogisticRegression(C=1.5, penalty='l1', solver='liblinear')
lr_l1_model1 = lr_l1_SF.fit(X_train1, y_train1)
lr_l1_ypred1 = lr_l1_model1.predict(X_test1)


lr_l1_cm_1 = confusion_matrix(y_test1, lr_l1_ypred1, labels=lr_l1_SF.classes_)
lr_l1_cm_1 = pd.DataFrame(lr_l1_cm_1, columns=lr_l1_SF.classes_, index=lr_l1_SF.classes_) #lr_l1_SF is instance of model
lr_l1_cm_1

In [ ]:
#adapt this!!!!!!
lr_l2_SF = LogisticRegression(C=1.5, penalty='l2', solver='liblinear')
lr_l2_model1 = lr_l2_SF.fit(X_train1, y_train1)
lr_l2_ypred1 = lr_l2_model1.predict(X_test1)
lr_l2_cm_2 = confusion_matrix(y_test1, lr_l2_ypred1, labels=lr_l2_SF.classes_)
lr_l2_cm_2 = pd.DataFrame(lr_l2_cm_2, columns=lr_l2_SF.classes_, index=lr_l2_SF.classes_) #lr_l2_SF is instance of model
lr_l2_cm_2


logreg_cv1 = LogisticRegressionCV(Cs=20, solver='liblinear', cv=3, penalty='l1', scoring='f1')
cv_model1 = logreg_cv1.fit(X_train1, y_train1)
logreg1 = LogisticRegression(solver='liblinear')
C_vals = [0.0001, 0.001, 0.01, 0.1, .15, .25, .275, .33, 0.5, .66, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]
penalties = ['l1','l2']

gs1 = GridSearchCV(logreg1, {'penalty': penalties, 'C': C_vals}, verbose=False, cv=15)
gs1.fit(X1, y1)

gs1.best_params_

#

In [18]:
## YOUR CODE HERE

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [19]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [20]:
## YOUR CODE HERE

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [21]:
## YOUR CODE HERE

In [22]:
## YOUR CODE HERE

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [23]:
## YOUR CODE HERE

In [24]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [25]:
## YOUR CODE HERE